In [33]:
import random
import tkinter as tk
from tkinter import messagebox, filedialog

class SudokuGenerator:
    @staticmethod
    def generate_base_sudoku():
        while True:
            puzzle = [[0 for _ in range(9)] for _ in range(9)]
            numbers = list(range(1, 10))
            random.shuffle(numbers)

            if SudokuGenerator.fill_sudoku(puzzle, numbers):
                return puzzle

    @staticmethod
    def fill_sudoku(puzzle, numbers):
        row, col = SudokuGenerator.find_empty_location(puzzle)
        if row is None:
            return True

        for num in numbers:
            if SudokuGenerator.is_valid(puzzle, num, row, col):
                puzzle[row][col] = num

                if SudokuGenerator.fill_sudoku(puzzle, numbers):
                    return True

                puzzle[row][col] = 0
        return False

    @staticmethod
    def find_empty_location(puzzle):
        for row in range(9):
            for col in range(9):
                if puzzle[row][col] == 0:
                    return row, col
        return None, None

    @staticmethod
    def is_valid(puzzle, num, row, col):
        for i in range(9):
            if puzzle[row][i] == num:
                return False

        for i in range(9):
            if puzzle[i][col] == num:
                return False

        box_row = (row // 3) * 3
        box_col = (col // 3) * 3
        for i in range(3):
            for j in range(3):
                if puzzle[box_row + i][box_col + j] == num:
                    return False
        return True

    @staticmethod
    def generate_sudoku(difficulty):
        puzzle = SudokuGenerator.generate_base_sudoku()

        if difficulty == 1:
            num_to_remove = random.randint(35, 38)
        elif difficulty == 2:
            num_to_remove = random.randint(54, 57)
        else:
            num_to_remove = random.randint(73, 76)

        cells_removed = 0
        while cells_removed < num_to_remove:
            row = random.randint(0, 8)
            col = random.randint(0, 8)
            if puzzle[row][col] != 0:
                puzzle[row][col] = 0
                cells_removed += 1

        return puzzle

In [34]:
class CSP:
    def __init__(self, variables, domains, constraints):
        self.variables = variables
        self.domains = domains
        self.constraints = constraints
        self.solution = None

    def solve(self, steps=None):
        assignment = {}
        self.backtrack(assignment, steps)
        return self.solution

    def backtrack(self, assignment, steps=None):
        if len(assignment) == len(self.variables):
            self.solution = assignment.copy()
            if steps is not None:
                grid = [[0 for _ in range(9)] for _ in range(9)]
                for (i, j), val in assignment.items():
                    grid[i][j] = val
                steps.append(grid)
            return True

        var = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(var, assignment):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                if steps is not None:
                    steps.append(assignment.copy())

                if self.backtrack(assignment, steps):
                    return True
                del assignment[var]
                if steps is not None:
                    steps.append(assignment.copy())
        return False

    def select_unassigned_variable(self, assignment):
        unassigned_vars = [var for var in self.variables if var not in assignment]
        return min(unassigned_vars, key=lambda var: len(self.domains[var]))

    def order_domain_values(self, var, assignment):
        return self.domains[var]

    def is_consistent(self, var, value, assignment):
        for constraint_var in self.constraints[var]:
            if constraint_var in assignment and assignment[constraint_var] == value:
                return False
        return True

In [35]:
class SudokuGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sudoku Solver")

        self.puzzle = None
        self.solution = None
        self.solution_steps = []
        self.current_step = 0

        self.create_widgets()

    def create_widgets(self):
        # Input Frame
        self.input_frame = tk.LabelFrame(self.root, text="Sudoku Input", padx=10, pady=10)
        self.input_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

        self.entry_grid = [[None for _ in range(9)] for _ in range(9)]
        for i in range(9):
            for j in range(9):
                self.entry_grid[i][j] = tk.Entry(
                    self.input_frame, 
                    width=2, 
                    font=("Roboto", 20), 
                    justify="center",
                    highlightthickness=2,
                    highlightbackground="black" if (i in (3, 4, 5) or j in (3, 4, 5)) and not (i in (3, 4, 5) and j in (3, 4, 5)) else "gray"
                )
                self.entry_grid[i][j].grid(row=i, column=j, padx=1, pady=1)
        
        self.puzzle_index = tk.IntVar(value=0)

        self.puzzle_selection_frame = tk.LabelFrame(self.root, text="Puzzle Selection", padx=10, pady=5)
        self.puzzle_selection_frame.grid(row=2, column=0, padx=10, pady=5, sticky="nsew")

        self.puzzle_radio_buttons = []

        # Control Frame
        self.control_frame = tk.Frame(self.root)
        self.control_frame.grid(row=1, column=0, padx=10, pady=10)

        self.generate_button = tk.Button(self.control_frame, text="Generate Puzzle", command=self.generate_puzzle)
        self.generate_button.grid(row=0, column=0, padx=5, pady=5)

        self.difficulty_label = tk.Label(self.control_frame, text="Difficulty:")
        self.difficulty_label.grid(row=0, column=2, padx=5, pady=5)

        self.difficulty = tk.StringVar(value="2")
        self.difficulty_options = [
            ("Easy", "1"),
            ("Medium", "2"),
            ("Hard", "3")
        ]
        for text, val in self.difficulty_options:
            tk.Radiobutton(self.control_frame, text=text, variable=self.difficulty, value=val).grid(row=0, column=2 + int(val), padx=1, pady=5)

        self.load_button = tk.Button(self.control_frame, text="Load from Text", command=self.load_from_text)
        self.load_button.grid(row=1, column=0, padx=5, pady=5)

        self.solve_button = tk.Button(self.control_frame, text="Solve", command=self.solve_sudoku)
        self.solve_button.grid(row=1, column=0, columnspan=4, padx=5, pady=5)

        self.step_label = tk.Label(self.control_frame, text="Step: 0/0")
        self.step_label.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

        self.prev_button = tk.Button(self.control_frame, text="Previous", command=self.previous_step, state="disabled")
        self.prev_button.grid(row=3, column=0, padx=1, pady=1)

        self.next_button = tk.Button(self.control_frame, text="Next", command=self.next_step, state="disabled")
        self.next_button.grid(row=3, column=1, padx=1, pady=1)

        self.fast_forward_button = tk.Button(self.control_frame, text="Fast Forward", command=self.fast_forward)
        self.fast_forward_button.grid(row=3, column=3, padx=1, pady=1)

        self.stop_button = tk.Button(self.control_frame, text="Stop", command=self.stop_fast_forward, state="disabled")
        self.stop_button.grid(row=3, column=4, padx=1, pady=1)

    def load_from_text(self):
        file_path = filedialog.askopenfilename(
            defaultextension=".txt",
            filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
        )
        if file_path:
            try:
                self.puzzles = self.parse_sudoku_from_text(file_path)

                # Create/update radio buttons for puzzle selection
                for i, rb in enumerate(self.puzzle_radio_buttons):
                    rb.destroy()  # Remove old radio buttons
                self.puzzle_radio_buttons = []  # Clear the list
                for i, _ in enumerate(self.puzzles):
                    rb = tk.Radiobutton(
                        self.puzzle_selection_frame,
                        text=f"Puzzle {i + 1}",
                        variable=self.puzzle_index,
                        value=i,
                        command=self.select_puzzle
                    )
                    rb.grid(row=0, column=i, padx=2, pady=2)
                    self.puzzle_radio_buttons.append(rb)

                if self.puzzles:
                    self.select_puzzle()  # Load the first puzzle initially

            except FileNotFoundError:
                messagebox.showerror("Error", f"File not found: {file_path}")
            except ValueError as e:
                messagebox.showerror("Error", f"Invalid Sudoku format: {e}")

    def parse_sudoku_from_text(self, file_path):
        puzzles = []
        current_puzzle = []
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if not line and current_puzzle:
                    puzzles.append(current_puzzle)
                    current_puzzle = []
                elif line:
                    row = [int(num) if num.isdigit() else 0 for num in line.split()]
                    if len(row) != 9:
                        raise ValueError("Each row in the Sudoku puzzle must have 9 elements.")
                    current_puzzle.append(row)
            if current_puzzle:
                puzzles.append(current_puzzle)
        return puzzles
    
    def select_puzzle(self):
        self.puzzle = self.puzzles[self.puzzle_index.get()]
        self.solution = None
        self.solution_steps = []
        self.current_step = 0
        self.update_grid()
        self.update_step_label()
        self.prev_button.config(state="disabled")
        self.next_button.config(state="disabled")

    def generate_puzzle(self):
        difficulty = int(self.difficulty.get())
        self.puzzle = SudokuGenerator.generate_sudoku(difficulty)
        self.solution = None
        self.solution_steps = []
        self.current_step = 0
        self.update_grid()
        self.update_step_label()
        self.prev_button.config(state="disabled")
        self.next_button.config(state="disabled")

    def solve_sudoku(self):
        if self.puzzle is not None:
            self.solution_steps = []
            self.current_step = 0
            variables = [(i, j) for i in range(9) for j in range(9)]
            domains = {var: set(range(1, 10)) if self.puzzle[var[0]][var[1]] == 0
                                else {self.puzzle[var[0]][var[1]]} for var in variables}

            def add_constraint(var):
                constraints[var] = []
                for i in range(9):
                    if i != var[0]:
                        constraints[var].append((i, var[1]))
                    if i != var[1]:
                        constraints[var].append((var[0], i))
                sub_i, sub_j = var[0] // 3, var[1] // 3
                for i in range(sub_i * 3, (sub_i + 1) * 3):
                    for j in range(sub_j * 3, (sub_j + 1) * 3):
                        if (i, j) != var:
                            constraints[var].append((i, j))

            constraints = {}
            for i in range(9):
                for j in range(9):
                    add_constraint((i, j))

            csp = CSP(variables, domains, constraints)
            csp.solve(self.solution_steps)
            self.solution = self.solution_steps[-1] if self.solution_steps else None

            if self.solution:
                self.update_grid()
                self.update_step_label()
                self.next_button.config(state="normal" if self.current_step < len(self.solution_steps) - 1 else "disabled")
                self.prev_button.config(state="normal" if self.current_step > 0 else "disabled")
            else:
                messagebox.showinfo("Sudoku Solver", "No solution found!")

    def previous_step(self):
        if self.current_step > 0:
            self.current_step -= 1
            self.update_grid()
            self.update_step_label()
            self.next_button.config(state="normal")
            self.prev_button.config(state="normal" if self.current_step > 0 else "disabled")

    def next_step(self):
        if self.current_step < len(self.solution_steps) - 1:
            self.current_step += 1
            self.update_grid()
            self.update_step_label()
            self.prev_button.config(state="normal")
            self.next_button.config(state="normal" if self.current_step < len(self.solution_steps) - 1 else "disabled")
    
    def fast_forward(self):
        if self.solution_steps:
            self.fast_forward_button.config(state="disabled")
            self.animate_solution(self.current_step + 1)
            self.stop_button.config(state="normal")
            self.next_button.config(state="disabled")
            self.prev_button.config(state="normal")

    def stop_fast_forward(self):
        self.stop_button.config(state="disabled")  
        self.fast_forward_button.config(state="normal") 
        if self.animation_job:
            self.root.after_cancel(self.animation_job)  

    def animate_solution(self, step):
        if step < len(self.solution_steps):
            self.current_step = step
            self.update_grid()
            self.update_step_label()
            self.animation_job = self.root.after(25, self.animate_solution, step + 1)
        else:
            self.fast_forward_button.config(state="normal")
            self.stop_button.config(state="disabled")
            self.animation_job = None

    def update_grid(self):
        if self.current_step == 0:
            data_to_display = self.puzzle 
        else:
            data_to_display = self.solution_steps[self.current_step - 1]

        for i in range(9):
            for j in range(9):
                if self.current_step == 0:
                    if data_to_display[i][j] != 0:
                        self.entry_grid[i][j].config(state="normal")
                        self.entry_grid[i][j].delete(0, tk.END)
                        self.entry_grid[i][j].insert(0, str(data_to_display[i][j]))
                        self.entry_grid[i][j].config(state="readonly")
                    else:
                        self.entry_grid[i][j].config(state="normal")
                        self.entry_grid[i][j].delete(0, tk.END)
                        self.entry_grid[i][j].config(state="readonly")
                else:
                    if (i, j) in data_to_display:  
                        self.entry_grid[i][j].config(state="normal")
                        self.entry_grid[i][j].delete(0, tk.END)
                        self.entry_grid[i][j].insert(0, str(data_to_display[(i, j)]))
                        self.entry_grid[i][j].config(state="readonly")
                    else:
                        self.entry_grid[i][j].config(state="normal")
                        self.entry_grid[i][j].delete(0, tk.END)
                        self.entry_grid[i][j].config(state="readonly")

    def update_step_label(self):
        self.step_label.config(text=f"Step: {self.current_step}/{len(self.solution_steps)}")

In [36]:
if __name__ == "__main__":
    root = tk.Tk()
    sudoku_gui = SudokuGUI(root)
    root.mainloop()